<a href="https://colab.research.google.com/github/NeuromatchAcademy/course-content-dl/blob/w3d5_updates/tutorials/W3D5_ReinforcementLearningForGames/W3D5_Tutorial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 2: Deep Learning Thinking 3

**Week 3, Day 5: Deep Learning Thinking 3 & Reinforcement Learning for Games**

**By Neuromatch Academy**


__Content creators:__ Konrad Kording, Lyle Ungar

__Content reviewers:__ Ella Batty

__Content editors:__ Ella Batty

__Production editors:__ Ella Batty, Spiros Chavlis

<p align='center'><img src='https://github.com/NeuromatchAcademy/widgets/blob/master/sponsors.png?raw=True'/></p>

---
# Tutorial Objectives



---
# Setup

In [ ]:
# @title Install dependencies
!pip3 install vibecheck datatops --quiet

from vibecheck import DatatopsContentReviewContainer

def content_review(notebook_section: str):
    return DatatopsContentReviewContainer(
        "",  # No text prompt
        notebook_section,
        {
            "url": "https://pmyvdlilci.execute-api.us-east-1.amazonaws.com/klab",
            "name": "neuromatch_dl",
            "user_key": "f379rz8y",
        },
    ).render()

feedback_prefix = "W3D5_T2"

---
# Section 1: Intro to Deep Learning Thinking 3

*Time estimate: ~3 mins*

In [ ]:
# @title Video 1: Intro to DL Thinking 3
from ipywidgets import widgets
from IPython.display import YouTubeVideo
from IPython.display import IFrame
from IPython.display import display


class PlayVideo(IFrame):
  def __init__(self, id, source, page=1, width=400, height=300, **kwargs):
    self.id = id
    if source == 'Bilibili':
      src = f'https://player.bilibili.com/player.html?bvid={id}&page={page}'
    elif source == 'Osf':
      src = f'https://mfr.ca-1.osf.io/render?url=https://osf.io/download/{id}/?direct%26mode=render'
    super(PlayVideo, self).__init__(src, width, height, **kwargs)


def display_videos(video_ids, W=400, H=300, fs=1):
  tab_contents = []
  for i, video_id in enumerate(video_ids):
    out = widgets.Output()
    with out:
      if video_ids[i][0] == 'Youtube':
        video = YouTubeVideo(id=video_ids[i][1], width=W,
                             height=H, fs=fs, rel=0)
        print(f'Video available at https://youtube.com/watch?v={video.id}')
      else:
        video = PlayVideo(id=video_ids[i][1], source=video_ids[i][0], width=W,
                          height=H, fs=fs, autoplay=False)
        if video_ids[i][0] == 'Bilibili':
          print(f'Video available at https://www.bilibili.com/video/{video.id}')
        elif video_ids[i][0] == 'Osf':
          print(f'Video available at https://osf.io/{video.id}')
      display(video)
    tab_contents.append(out)
  return tab_contents


video_ids = [('Youtube', 'rURUYJUKJQY')]
tab_contents = display_videos(video_ids, W=854, H=480)
tabs = widgets.Tab()
tabs.children = tab_contents
for i in range(len(tab_contents)):
  tabs.set_title(i, video_ids[i][0])
display(tabs)

This tutorial is the third installment of our deep learning thinking series. Like the others, there will be no coding! Instead, you will watch a series of vignettes about various scenarios where you want to use a neural network.


Each section below will start with a vignette where either Lyle or Konrad is trying to figure out how to set up a neural network for a specific problem. Try to think of questions you want to ask them as you watch, then pay attention to what questions Lyle and Konrad are asking. Were they what you would have asked? How do their questions help quickly clarify the situation?

In [ ]:
# @title Submit your feedback
content_review(f"{feedback_prefix}_Intro_Video")

---
# Section 2: The Future

*Time estimate: ~15 mins*

In [ ]:
# @title Video 2: The Future
from ipywidgets import widgets
from IPython.display import YouTubeVideo
from IPython.display import IFrame
from IPython.display import display


class PlayVideo(IFrame):
  def __init__(self, id, source, page=1, width=400, height=300, **kwargs):
    self.id = id
    if source == 'Bilibili':
      src = f'https://player.bilibili.com/player.html?bvid={id}&page={page}'
    elif source == 'Osf':
      src = f'https://mfr.ca-1.osf.io/render?url=https://osf.io/download/{id}/?direct%26mode=render'
    super(PlayVideo, self).__init__(src, width, height, **kwargs)


def display_videos(video_ids, W=400, H=300, fs=1):
  tab_contents = []
  for i, video_id in enumerate(video_ids):
    out = widgets.Output()
    with out:
      if video_ids[i][0] == 'Youtube':
        video = YouTubeVideo(id=video_ids[i][1], width=W,
                             height=H, fs=fs, rel=0)
        print(f'Video available at https://youtube.com/watch?v={video.id}')
      else:
        video = PlayVideo(id=video_ids[i][1], source=video_ids[i][0], width=W,
                          height=H, fs=fs, autoplay=False)
        if video_ids[i][0] == 'Bilibili':
          print(f'Video available at https://www.bilibili.com/video/{video.id}')
        elif video_ids[i][0] == 'Osf':
          print(f'Video available at https://osf.io/{video.id}')
      display(video)
    tab_contents.append(out)
  return tab_contents


video_ids = [('Youtube', 'fqdbgkimsxo')]
tab_contents = display_videos(video_ids, W=854, H=480)
tabs = widgets.Tab()
tabs.children = tab_contents
for i in range(len(tab_contents)):
  tabs.set_title(i, video_ids[i][0])
display(tabs)

## Think! 1: The Future

Things change over time! How might we address this problem in machine learning?

Please discuss as a group. If you get stuck, you can uncover the hints below one at a time to help guide your discussion. Please spend some time discussing before uncovering the next hint though!

<details>
<summary> <font color='green'>Click here for hint 1 </font></summary>

Can you think of a scenario in your life where data is i.i.d?

Let's use an example. Konrad hears questions x about a topic during a class he's teaching, and tries to learn answers y to each of them. Will the questions next year be the same as the questions this year?

Once we know that the questions next year will be different, what should Konrad do to be ready for the questions next year?

More generally, why does it matter that the future is different from the past? How does that relate to i.i.d.?


<details>
<summary> <font color='green'>Click here for hint 2 </font></summary>

So the future will have a different distribution p(X) than the past. It will also have a different distribution of good answers p(Y). After all, deep learning is evolving. And training in deep learning is also evolving. What could be a good model, in the context of deep learning, of p(X)? How could it be built into a deep learning system? Would it even be necessary to build it in? What about p(Y)? Would that need to be built in? Arguably, DL is about how to find p(Y|X). Why do we need to care about X changing? Why about Y changing?

In this context, let us talk about curiosity. If p(X) and p(Y) are both changing, what would we want to learn about? We probably want to focus on things that will still be true in the future. What are the things that are true today and will be true in the future? Which things change?


<details>
<summary> <font color='green'>Click here for hint 3 </font></summary>

The objects in a room change. Their affordances do not, you will be able to sit on a chair tomorrow. The causal relations in the world do not change. For example, fire will still burn wood in the future. Their constituents do not change. For example, mammals still consist of blood, flesh, and bones. Other things change all the time. Who is in this room? Who am I talking with? There are also things that do change that I do not care about, e.g. somewhere far away there is a causal system I do not understand. So we do not just want to have good models in the future, we want to have good models to answer the kinds of questions we will actually be asked.


<details>
<summary> <font color='green'>Click here for solution  </font></summary>

Here is a paper talking about these phenomena: https://arxiv.org/abs/2201.07372.

The paper above is worth a full read! In brief, it outlines an argument that most AI focuses on *retrospective learning*, where it uses past experiences to learn and make predictions in the future, assuming the future mimics the past. The authors argue that true intelligence is *prospective learning*, where AI learns for an uncertain future by updating internal models to be useful for future novel tasks. They articulate four revelant factors that jointly define prospective learning: "**Continual learning** enables intelligences to remember those aspects of the past which
it believes will be most useful in the future. Prospective **constraints** (including biases and priors) facilitate the
intelligence finding general solutions that will be applicable to future problems. **Curiosity** motivates taking actions
that inform future decision making, including in previously unmet situations. **Causal estimation** enables learning
the structure of relations that guide choosing actions for specific outcomes, even when the specific action-outcome
contingencies have never been observed before."


In [ ]:
# @title Submit your feedback
content_review(f"{feedback_prefix}_The_Future_Think_Video")

---
# Section 3: In-context Learning

*Time estimate: ~15 mins*

In [ ]:
# @title Video 3: In-context Learning Vignette
from ipywidgets import widgets
from IPython.display import YouTubeVideo
from IPython.display import IFrame
from IPython.display import display


class PlayVideo(IFrame):
  def __init__(self, id, source, page=1, width=400, height=300, **kwargs):
    self.id = id
    if source == 'Bilibili':
      src = f'https://player.bilibili.com/player.html?bvid={id}&page={page}'
    elif source == 'Osf':
      src = f'https://mfr.ca-1.osf.io/render?url=https://osf.io/download/{id}/?direct%26mode=render'
    super(PlayVideo, self).__init__(src, width, height, **kwargs)


def display_videos(video_ids, W=400, H=300, fs=1):
  tab_contents = []
  for i, video_id in enumerate(video_ids):
    out = widgets.Output()
    with out:
      if video_ids[i][0] == 'Youtube':
        video = YouTubeVideo(id=video_ids[i][1], width=W,
                             height=H, fs=fs, rel=0)
        print(f'Video available at https://youtube.com/watch?v={video.id}')
      else:
        video = PlayVideo(id=video_ids[i][1], source=video_ids[i][0], width=W,
                          height=H, fs=fs, autoplay=False)
        if video_ids[i][0] == 'Bilibili':
          print(f'Video available at https://www.bilibili.com/video/{video.id}')
        elif video_ids[i][0] == 'Osf':
          print(f'Video available at https://osf.io/{video.id}')
      display(video)
    tab_contents.append(out)
  return tab_contents


video_ids = [('Youtube', 'V5n41NHMbZI')]
tab_contents = display_videos(video_ids, W=854, H=480)
tabs = widgets.Tab()
tabs.children = tab_contents
for i in range(len(tab_contents)):
  tabs.set_title(i, video_ids[i][0])
display(tabs)

## Think! 2: In-context Learning

Please discuss as a group. If you get stuck, you can uncover the hints below one at a time to guide your discussion. Please spend some time discussing before uncovering the next hint though!

<details>
<summary> <font color='green'>Click here for hint 1 </font></summary>

What is in context learning? The context is basically the words that LLMs have in their context right now. To predicts what is next we, arguably, solve a machine learning problem. How would you write this as an equation?

Try if you can get chatGPT to solve in-context learning problems.


<details>
<summary> <font color='green'>Click here for hint 2 </font></summary>

Let us talk about the nature of the problem. How to we train systems and how may that relate to how we do in-context learning?

Meta-learning is defined as a system that learns how to learn. Is in-context learning a version of meta-learning?


<details>
<summary> <font color='green'>Click here for hint 3 </font></summary>

How could the transformer architecture be helpful for solving in-context learning problems? DL does do gradient descent. Do you think there is gradient descent learning in-context?


<details>
<summary> <font color='green'>Click here for solution  </font></summary>

Here are two papers about these topics:
https://arxiv.org/abs/2211.15661
https://arxiv.org/abs/2212.10559v2



In [ ]:
# @title Submit your feedback
content_review(f"{feedback_prefix}_In_context_learning_Think_Video")

---
# Section 4: Memories


*Time estimate: ~15 mins*

In [ ]:
# @title Video 4: Memories Vignette
from ipywidgets import widgets
from IPython.display import YouTubeVideo
from IPython.display import IFrame
from IPython.display import display


class PlayVideo(IFrame):
  def __init__(self, id, source, page=1, width=400, height=300, **kwargs):
    self.id = id
    if source == 'Bilibili':
      src = f'https://player.bilibili.com/player.html?bvid={id}&page={page}'
    elif source == 'Osf':
      src = f'https://mfr.ca-1.osf.io/render?url=https://osf.io/download/{id}/?direct%26mode=render'
    super(PlayVideo, self).__init__(src, width, height, **kwargs)


def display_videos(video_ids, W=400, H=300, fs=1):
  tab_contents = []
  for i, video_id in enumerate(video_ids):
    out = widgets.Output()
    with out:
      if video_ids[i][0] == 'Youtube':
        video = YouTubeVideo(id=video_ids[i][1], width=W,
                             height=H, fs=fs, rel=0)
        print(f'Video available at https://youtube.com/watch?v={video.id}')
      else:
        video = PlayVideo(id=video_ids[i][1], source=video_ids[i][0], width=W,
                          height=H, fs=fs, autoplay=False)
        if video_ids[i][0] == 'Bilibili':
          print(f'Video available at https://www.bilibili.com/video/{video.id}')
        elif video_ids[i][0] == 'Osf':
          print(f'Video available at https://osf.io/{video.id}')
      display(video)
    tab_contents.append(out)
  return tab_contents


video_ids = [('Youtube', 'Sr2T9VZlNJY'), ('Bilibili', '')]
tab_contents = display_videos(video_ids, W=854, H=480)
tabs = widgets.Tab()
tabs.children = tab_contents
for i in range(len(tab_contents)):
  tabs.set_title(i, video_ids[i][0])
display(tabs)

## Think! 3: Memories

Please discuss as a group. If you get stuck, you can uncover the hints below one at a time. Please spend some time discussing before uncovering the next hint, though! You are being real deep learning scientists now, and the answers won’t be easy.



<details>
<summary> <font color='green'>Click here for hint 1 </font></summary>

How would you define episodic memory?

How would you define procedural memory?

How would you define semantic memory?


<details>
<summary> <font color='green'>Click here for hint 2 </font></summary>

The main interesting thing here is episodic memory. For example, you and your best friend may both know that it is a fact that you had dinner together yesterday night. How would you build episodic memory into a deep learning systems? Can you gradient descent into an episodic memory?



<details>
<summary> <font color='green'>Click here for hint 3 </font></summary>

Episodic memory is after all something about absolute truth. So it is not entirely clear how it relates to gradient descent. It is easy to see how we may use gradient descent for reading though. So what if you instead just had a big blackboard. What would you write on it? When/how would you erase it?

<details>
<summary> <font color='green'>Click here for solution </font></summary>

Here are a few fun papers. https://arxiv.org/abs/1410.5401, https://arxiv.org/abs/1805.07603, https://arxiv.org/abs/1703.03129

In [ ]:
# @title Submit your feedback
content_review(f"{feedback_prefix}_Memories_Think_Video")

---
# Section 5: Multiple Information Sources


*Time estimate: ~15 mins*

In [ ]:
# @title Video 5: Multiple Information Sources Vignette
from ipywidgets import widgets
from IPython.display import YouTubeVideo
from IPython.display import IFrame
from IPython.display import display


class PlayVideo(IFrame):
  def __init__(self, id, source, page=1, width=400, height=300, **kwargs):
    self.id = id
    if source == 'Bilibili':
      src = f'https://player.bilibili.com/player.html?bvid={id}&page={page}'
    elif source == 'Osf':
      src = f'https://mfr.ca-1.osf.io/render?url=https://osf.io/download/{id}/?direct%26mode=render'
    super(PlayVideo, self).__init__(src, width, height, **kwargs)


def display_videos(video_ids, W=400, H=300, fs=1):
  tab_contents = []
  for i, video_id in enumerate(video_ids):
    out = widgets.Output()
    with out:
      if video_ids[i][0] == 'Youtube':
        video = YouTubeVideo(id=video_ids[i][1], width=W,
                             height=H, fs=fs, rel=0)
        print(f'Video available at https://youtube.com/watch?v={video.id}')
      else:
        video = PlayVideo(id=video_ids[i][1], source=video_ids[i][0], width=W,
                          height=H, fs=fs, autoplay=False)
        if video_ids[i][0] == 'Bilibili':
          print(f'Video available at https://www.bilibili.com/video/{video.id}')
        elif video_ids[i][0] == 'Osf':
          print(f'Video available at https://osf.io/{video.id}')
      display(video)
    tab_contents.append(out)
  return tab_contents


video_ids = [('Youtube', 'v76xc1jzaPU')]
tab_contents = display_videos(video_ids, W=854, H=480)
tabs = widgets.Tab()
tabs.children = tab_contents
for i in range(len(tab_contents)):
  tabs.set_title(i, video_ids[i][0])
display(tabs)

## Think! 4: Multiple Information Sources

Please discuss as a group. If you get stuck, you can uncover the hints below one at a time. Please spend some time discussing before uncovering the next hint, though! You are being real deep learning scientists now, and the answers won’t be easy.

<details>
<summary> <font color='green'>Click here for hint 1 </font></summary>

Think of a few tasks. If you wanted to solve them, which webpages would you use? How would you combine them? How could you allow a DL system to do the same thing?


<details>
<summary> <font color='green'>Click here for hint 2 </font></summary>

If you have multiple sources of information, how synergistic are they? How could you incorporate information from multiple sources? Could there be ways of summarizing the things you could get out of a given model?

<details>
<summary> <font color='green'>Click here for hint 3 </font></summary>

English language and querying can be a way of interacting with such services. Think about webpages as something that you can ask questions to.


<details>
<summary> <font color='green'>Click here for solution </font></summary>

Here are a list of recent relevant projects
https://github.com/Significant-Gravitas/Auto-GPT
https://arxiv.org/abs/2302.14045, https://ai.googleblog.com/2023/03/palm-e-embodied-multimodal-language.html

In [ ]:
# @title Submit your feedback
content_review(f"{feedback_prefix}_MultipleInformationSources_Think_Video")

---
# Section 6: Language for Robotics


*Time estimate: ~15 mins*

In [ ]:
# @title Video 6: Language for Robotics Vignette
from ipywidgets import widgets
from IPython.display import YouTubeVideo
from IPython.display import IFrame
from IPython.display import display


class PlayVideo(IFrame):
  def __init__(self, id, source, page=1, width=400, height=300, **kwargs):
    self.id = id
    if source == 'Bilibili':
      src = f'https://player.bilibili.com/player.html?bvid={id}&page={page}'
    elif source == 'Osf':
      src = f'https://mfr.ca-1.osf.io/render?url=https://osf.io/download/{id}/?direct%26mode=render'
    super(PlayVideo, self).__init__(src, width, height, **kwargs)


def display_videos(video_ids, W=400, H=300, fs=1):
  tab_contents = []
  for i, video_id in enumerate(video_ids):
    out = widgets.Output()
    with out:
      if video_ids[i][0] == 'Youtube':
        video = YouTubeVideo(id=video_ids[i][1], width=W,
                             height=H, fs=fs, rel=0)
        print(f'Video available at https://youtube.com/watch?v={video.id}')
      else:
        video = PlayVideo(id=video_ids[i][1], source=video_ids[i][0], width=W,
                          height=H, fs=fs, autoplay=False)
        if video_ids[i][0] == 'Bilibili':
          print(f'Video available at https://www.bilibili.com/video/{video.id}')
        elif video_ids[i][0] == 'Osf':
          print(f'Video available at https://osf.io/{video.id}')
      display(video)
    tab_contents.append(out)
  return tab_contents


video_ids = [('Youtube', 'By7oLXsCh0w')]
tab_contents = display_videos(video_ids, W=854, H=480)
tabs = widgets.Tab()
tabs.children = tab_contents
for i in range(len(tab_contents)):
  tabs.set_title(i, video_ids[i][0])
display(tabs)

## Think! 5: Language for Robotics

<details>
<summary> <font color='green'>Click here for hint 1 </font></summary>

Think about a robotics problem, e.g. emptying the dishwasher. Is there a way of dividing it into subproblems? How many of them? How complex are they?


<details>
<summary> <font color='green'>Click here for hint 2 </font></summary>

If you look at the subproblems, how complex are they? What could be a good way of describing each subproblem? If a robot controlling AI system would have such a description, how useful would that be?

<details>
<summary> <font color='green'>Click here for hint 3 </font></summary>

If we were to use a large language model to interact with the RL systems, how useful would that be? Why is language so useful for this task?

<details>
<summary> <font color='green'>Click here for solution </font></summary>

Have a look at this great collection of papers
https://github.com/GT-RIPL/Awesome-LLM-Robotics


In [ ]:
# @title Submit your feedback
content_review(f"{feedback_prefix}_LanguageForRobotics_Think_Video")

---
# Summary

*Time estimate: ~2 mins*



